In [34]:
!pip install -qU langchain_community beautifulsoup4


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
from langchain_groq import ChatGroq

In [61]:
llm = ChatGroq(
    temperature=0,
    groq_api_key ='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',
    model_name ="llama-3.1-70b-versatile"
)
response = llm.invoke("The First Person To Land On moon Was....")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [67]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-43849?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Software Engineer III, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer III, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc. d

In [68]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [69]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer III, ITC',
 'experience': '6-9 years',
 'skills': ['Java and Spring Boot framework',
  'Micro service architecture',
  'Domain driven design',
  'RESTful Services',
  'Test Driven development – with Junit, Mockito, cucumber',
  'Monitoring and tracing tools like Splunk, SignalFx, Kibana/Grafana',
  'NoSQL data stores like Cassandra, DynamoDB, MongoDB',
  'Relational data stores like MySQL, Oracle',
  'Continuous integration, unit testing, static analysis, and automated integration tests',
  'Scrum and agile principles'],
 'description': 'As a Software Engineer III for the Geographies & Marketplace Experiences team, you will be part of a mission critical team focused on building a high-throughput, low-latency, fault-tolerant supply chain platform that drive Nike’s Digital transformation and growth strategies.'}

In [70]:
type(json_res)

dict

In [71]:
import pandas as pd 

df=pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [72]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [75]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-port

In [76]:
job=json_res
job['skills']

['Java and Spring Boot framework',
 'Micro service architecture',
 'Domain driven design',
 'RESTful Services',
 'Test Driven development – with Junit, Mockito, cucumber',
 'Monitoring and tracing tools like Splunk, SignalFx, Kibana/Grafana',
 'NoSQL data stores like Cassandra, DynamoDB, MongoDB',
 'Relational data stores like MySQL, Oracle',
 'Continuous integration, unit testing, static analysis, and automated integration tests',
 'Scrum and agile principles']

In [77]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)


Subject: Expert Software Engineering Services for Nike's Digital Transformation

Dear Hiring Manager,

I came across the job description for a Software Engineer III, ITC at Nike, and I was impressed by the company's commitment to building a high-throughput, low-latency, fault-tolerant supply chain platform. As a Business Development Executive at AtliQ, I believe our team can help you achieve this goal.

At AtliQ, we specialize in providing AI and software consulting services that facilitate seamless integration of business processes through automated tools. Our expertise in Java and Spring Boot framework, microservice architecture, domain-driven design, and RESTful services aligns perfectly with the requirements of the job.

Our team has extensive experience in developing scalable and efficient software solutions using NoSQL data stores like Cassandra, DynamoDB, and MongoDB, as well as relational data stores like MySQL and Oracle. We also have expertise in monitoring and tracing tools 